In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [29]:
from typing import Literal, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from pydantic import BaseModel, Field

# Define MessagesState as a TypedDict for state management
class MessagesState(TypedDict):
    messages: list[BaseMessage]

# Define the Market Value Check response schema
class MarketValueCheckResponse(BaseModel):
    market_value_found: str = Field(
        description="Is the market value of the player mentioned in the document? Answer with 'Yes' or 'No'."
    )

# Define a fake tool for querying a "database" of market values
@tool
def get_market_value(player_name: str):
    """Fake tool to simulate querying a database for a player's market value."""
    fake_db = {
        "Lionel Messi": "Market value is 50 million Euros.",
        "Cristiano Ronaldo": "Market value is 40 million Euros.",
    }
    return fake_db.get(player_name, "Market value information not available.")

# Initialize model and bind tools
tools = [get_market_value]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

# Define the main function for the Market Value Researcher agent
def market_value_researcher(state: MessagesState):
    # Define the system message to guide the LLM's response
    system_message = SystemMessage(content="""You are an agent tasked with determining whether a document mentions the market value of a player.
    Only respond with 'Yes' if the document explicitly includes a player's market value, otherwise respond with 'No'.""")

    # Define the structured output using Pydantic for expected response format
    structured_model = model.with_structured_output(MarketValueCheckResponse)

    # Assume we get the player's name from state for querying the tool
    player_name = "Lionel Messi"  # Replace with dynamic input if available
    human_message = HumanMessage(content=f"Is the market value of {player_name} mentioned in the document?")

    # Invoke the model with the structured prompt
    response = structured_model.invoke({"messages": [system_message, human_message]})
    return {"messages": [response]}

# Build the workflow graph
market_value_graph = StateGraph(MessagesState)
market_value_graph.add_node("market_value_researcher", market_value_researcher)
market_value_graph.add_edge(START, "market_value_researcher")
market_value_graph.add_edge("market_value_researcher", END)

# Compile the graph for the Market Value Researcher
market_value_researcher_agent = market_value_graph.compile()


In [30]:
market_value_researcher_agent.invoke({"messages": [HumanMessage(content="Lionel Messi goes to Real Madrid in 2025")]})

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [ ]:
from typing import Literal, Annotated
from operator import add
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

class MessagesState(TypedDict):
    messages: Annotated[list[BaseMessage], add]


@tool
def get_current_club(player_name: str):
    """Fake tool to simulate querying a database for a player's current club."""
    fake_db = {
        "Lionel Messi": "Paris Saint-Germain",
        "Cristiano Ronaldo": "Al Nassr FC",
    }
    return fake_db.get(player_name, "Current club information not available.")

tools2 = [get_current_club]
model2 = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools2)

def call_model_current_club(state: MessagesState):
    messages = state['messages']
    system_message = SystemMessage(content="""You are an agent tasked with determining the current club of a player.
    If the current club is mentioned, return it. Otherwise, return 'Current club information not available.'""")
    response = model2.invoke([system_message] + messages)
    return {"messages": [response]}

def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

current_club_graph = StateGraph(MessagesState)
current_club_graph.add_node("call_model_current_club", call_model_current_club)
current_club_graph.add_node("tools", ToolNode(tools2))
current_club_graph.add_edge(START, "call_model_current_club")
current_club_graph.add_conditional_edges("call_model_current_club", should_continue)
current_club_graph.add_edge("tools", "call_model_current_club")

current_club_researcher_agent = current_club_graph.compile()


In [53]:
initial_state = {
    "article": "Lionel Messi will join Real Madrid 2025",
}
output = current_club_researcher_agent.invoke(initial_state)
current_club_researcher_agent.invoke(initial_state)

{'article': 'Lionel Messi will join Real Madrid 2025',
 'messages': [HumanMessage(content='Lionel Messi will join Real Madrid 2025', additional_kwargs={}, response_metadata={})],
 'lastmsg': ''}

In [25]:
from typing import Literal
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool




# Define a fake tool for querying a "database" of players' current clubs
@tool
def get_current_club(player_name: str):
    """Fake tool to simulate querying a database for a player's current club."""
    fake_db = {
        "Lionel Messi": "Paris Saint-Germain",
        "Cristiano Ronaldo": "Al Nassr FC",
    }
    return fake_db.get(player_name, "Current club information not available.")

# Initialize model and bind tools
tools = [get_current_club]
model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

# Define a function to call the model for Current Club Researcher
def call_model_current_club(state: MessagesState):
    messages = state['messages']
    # Prepend the system message for instructions
    system_message = SystemMessage(content="""You are an agent tasked with determining the current club of a player.
    If the current club is mentioned, return it. Otherwise, return 'Current club information not available.'""")
    # Invoke the model with the prepended system message
    response = model.invoke([system_message] + messages)
    return {"messages": [response]}

# Define a function to decide whether to continue using tools
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

# Build the workflow graph for the Current Club Researcher agent
current_club_graph = StateGraph(MessagesState)
current_club_graph.add_node("call_model_current_club", call_model_current_club)
current_club_graph.add_node("tools", ToolNode(tools))  # Using ToolNode for the tools
current_club_graph.add_edge(START, "call_model_current_club")
current_club_graph.add_conditional_edges("call_model_current_club", should_continue)
current_club_graph.add_edge("tools", "call_model_current_club")

# Compile the graph
current_club_researcher_agent = current_club_graph.compile()

In [26]:
initial_state = {"message": [HumanMessage(content="Lionel Messi joins a new club in Paris")]}
current_club_researcher_agent.invoke(initial_state)

InvalidUpdateError: Must write to at least one of ['messages']

### Supervisor Agent

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class ArticlePostabilityGrader(BaseModel):
    """Binary scores for verifying if an article mentions market value, current club, and meets the minimum word count of 100 words."""

    mentions_market_value: str = Field(
        description="The article mentions the player's market value, 'yes' or 'no'"
    )
    mentions_current_club: str = Field(
        description="The article mentions the player's current club, 'yes' or 'no'"
    )
    meets_100_words: str = Field(
        description="The article has at least 100 words, 'yes' or 'no'"
    )

llm_postability = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm_postability_grader = llm_postability.with_structured_output(
    ArticlePostabilityGrader
)

postability_system = """You are a grader assessing whether a news article meets the following criteria:
1. Mentions the player's market value.
2. Mentions the player's current club.
3. Contains at least 100 words.
Provide three binary scores: one indicating if the article mentions the player's market value ('yes' or 'no'), one for mentioning the player's current club ('yes' or 'no'), and one for meeting the 100-word count ('yes' or 'no')."""
postability_grade_prompt = ChatPromptTemplate.from_messages(
    [("system", postability_system), ("human", "News Article:\n\n{article}")]
)

news_chef = postability_grade_prompt | structured_llm_postability_grader

In [ ]:
news_chef.invoke({"article": "Messi will switch from FC Barcelona to Real Madrid in 2025"})

### Workflow

In [ ]:
from langchain_core.messages import BaseMessage, HumanMessage
from typing import TypedDict, Literal, List, Annotated
from operator import add

class SharedArticleState(TypedDict):
    article: str
    mentions_market_value: str
    mentions_current_club: str
    meets_100_words: str
    final_article: str


def update_article_state(state: SharedArticleState) -> SharedArticleState:
    result = news_chef.invoke({"article": state["article"]})

    state["mentions_market_value"] = result.mentions_market_value
    state["mentions_current_club"] = result.mentions_current_club
    state["meets_100_words"] = result.meets_100_words

    print("State after update_article_state:", state)
    return state


def market_value_researcher_node(state: SharedArticleState) -> SharedArticleState:
    response = market_value_researcher_agent.invoke({"messages": HumanMessage(content=state["messages"])})
    print("RESPONSE from market_value_researcher_node", response)
    state["mentions_market_value"] = response.mentions_market_value
    state["messages"].append(response[-1])
    return state

def current_club_researcher_node(state: SharedArticleState) -> SharedArticleState:
    response = current_club_researcher_agent.invoke({"messages": state["messages"]})
    print("RESPONSE from current_club_researcher_node", response)
    state["mentions_current_club"] = response.mentions_current_club
    state["messages"].append(response[-1])
    return state

def word_count_rewriter_node(state: SharedArticleState) -> SharedArticleState:
    response = word_count_rewriter_agent.invoke({"messages": state["messages"]})
    state["final_article"] = response.content
    print("RESPONSE from word_count_rewriter_node", response)
    state["meets_100_words"] = response.meets_100_words
    state["messages"].append(response[-1])
    return state

def news_chef_decider(state: SharedArticleState) -> Literal["market_value_researcher", "current_club_researcher", "word_count_rewriter", END]:
    if state["mentions_market_value"] == "no":
        return "market_value_researcher"
    elif state["mentions_current_club"] == "no":
        return "current_club_researcher"
    elif state["meets_100_words"] == "no" and state["mentions_market_value"] == "yes" and state["mentions_current_club"] == "yes":
        return "word_count_rewriter"
    else:
        return END

workflow = StateGraph(SharedArticleState)

workflow.add_node("news_chef", update_article_state)
workflow.add_node("market_value_researcher", market_value_researcher_agent)
workflow.add_node("current_club_researcher", current_club_researcher_agent)
workflow.add_node("word_count_rewriter", word_count_rewriter_agent)

workflow.set_entry_point("news_chef")

workflow.add_conditional_edges(
    "news_chef",
    news_chef_decider,
    {
        "market_value_researcher": "market_value_researcher",
        "current_club_researcher": "current_club_researcher",
        "word_count_rewriter": "word_count_rewriter",
        END: END
    },
)

workflow.add_edge("market_value_researcher", "news_chef")
workflow.add_edge("current_club_researcher", "news_chef")
workflow.add_edge("word_count_rewriter", "news_chef")

app = workflow.compile()

In [ ]:
sys_msg = SystemMessage(content=postability_system)
messages = sys_msg + [HumanMessage(content="Messi will switch from FC Barcelona to Real Madrid in 2025")]
result = app.invoke({"messages": messages.messages})

In [ ]:
result